In [1]:
#CODE TO GET CHANGES IN OVERALL AGENCY SPENDING
import pandas as pd
import os

In [13]:
#FEB 2024 CBO SPENDING PROJECTIONS
years = [i for i in range(2024, 2024+11)] 
cbo_ot = pd.read_excel('data/cbo-2024-02-Spending-Projections.xlsx', header=7, sheet_name=1)
cbo_ot = cbo_ot[[col for col in cbo_ot.columns if col not in years]] #remove the BA cols
cbo_ot.columns = cbo_ot.columns.str.replace('.1', '') #clean the OT cols
cbo_ot = cbo_ot.drop(columns=['Unnamed: 9', "Unnamed: 21", "Off-Budget?"]).drop(index=0).drop(cbo_ot.index[-2:]) #drop the weird empty rows and the empty cols and offbud col
cbo_ot['25-34'] = cbo_ot.loc[:, '2025':'2034'].sum(axis=1) #sum the ots over the 10yr window
print("Number of agencies in og cbo24 dataset:")
print(cbo_ot["Agency"].nunique())
cbo_ot["Agency"] = cbo_ot["Agency"].str.strip()
cbo_ot = cbo_ot[['Agency', "25-34"]].groupby("Agency").sum() #get sum of ots by agency
cbo_ot.head(3)

Number of agencies in og cbo24 dataset:
134


,25-34
Agency,
400 Years of African-American History Commission,32.0
Access Board,119.0
Administrative Conference of the United States,36.0


In [17]:
#FEB 2021 CBO SPENDING PROJECTIONS
cbo21_ot = pd.read_excel('data/cbo-2021-02-11-spendingprojections.xlsx', header=7, sheet_name=1)
years_21 = [i for i in range(2021, 2032)]
cbo21_ot = cbo21_ot[[col for col in cbo21_ot.columns if col not in years_21]].drop(columns=["Off-Budget?", "Unnamed: 9", "Unnamed: 21"]) #remove the BA cols, unnamed and unnecessary columns
cbo21_ot.columns = cbo21_ot.columns.str.replace('.1', '') #clean OT columns
cbo21_ot["22-31"] = cbo21_ot.loc[:, '2022':'2031'].sum(axis=1) #sum the years
print("Number of agencies in og cbo21 dataset:")
print(cbo21_ot["Agency"].nunique())
cbo21_ot["Agency"] = cbo21_ot["Agency"].str.strip()
cbo21_ot['Agency'] = cbo21_ot['Agency'].replace('Other Defense Civil Programs', 'Other Defense--Civil Programs').replace("Committee for Purchase from People Who Are Blind or Severely Dis", "Committee for Purchase From People Who Are Blind/Disabled" )

cbo21_ot = cbo21_ot[["Agency", "22-31"]].groupby("Agency").sum() #group by agency and sum the total
cbo21_ot.head(3)

Number of agencies in og cbo21 dataset:
132


,22-31
Agency,
Access Board,106
Administrative Conference of the United States,35
Advisory Council on Historic Preservation,82


In [18]:
#2024 AGENCIES NOT IN 2021 AGENCIES
for agency24 in cbo_ot.index:
    if agency24 not in cbo21_ot.index:
        print("This is a new agency since 2021:", agency24)

This is a new agency since 2021: 400 Years of African-American History Commission
This is a new agency since 2021: Southeast Crescent Regional Commission
This is a new agency since 2021: Southwest Border Regional Commission


In [19]:
#2021 AGENCIES NOT IN 2024 AGENCIES
for agency21 in cbo21_ot.index:
    if agency21 not in cbo_ot.index:
        print("This 2021 agency is no longer around:", agency21)

This 2021 agency is no longer around: United States Enrichment Corporation Fund


In [27]:
#MERGE
merged = pd.merge(cbo21_ot, cbo_ot, on="Agency", how="right")
merged['percent_change'] = ((merged['25-34'] - merged['22-31']) / abs(merged['22-31'])) * 100
merged['22-31'] = merged['22-31'].fillna('N/A')
merged['percent_change'] = merged['percent_change'].fillna('N/A')
merged.reset_index(inplace=True)
merged

,Agency,22-31,25-34,percent_change
0,400 Years of African-American History Commission,N/A,32.0,N/A
1,Access Board,106.0,119.0,12.264151
2,Administrative Conference of the United States,35.0,36.0,2.857143
3,Advisory Council on Historic Preservation,82.0,108.0,31.707317
4,Affordable Housing Program,3313.0,2506.0,-24.358587
...,...,...,...,...
129,United Mine Workers of America Benefit Funds,86.0,5072.0,5797.674419
130,United States Court of Appeals for Veterans Cl...,447.0,589.0,31.767338
131,United States Holocaust Memorial Museum,692.0,734.0,6.069364
132,United States Institute of Peace,497.0,619.0,24.547284


In [26]:
inf_values = merged[merged['percent_change'].isin([float('inf'), float('-inf')])]
inf_values

,Agency,22-31,25-34,percent_change
5,Allowances,0.0,-107233.0,-inf
21,Council of the Inspectors General on Integrity...,0.0,18.0,inf
61,Federal Retirement Thrift Investment Board,0.0,-264.0,-inf


In [29]:
sheetname = "agency increases"
filename = "output/cbo_projection_changes.xlsx"
df = merged
with pd.ExcelWriter(filename, mode='a', engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name=sheetname, index=False)